In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 8.6 MB/s 
     |████████████████████████████████| 486 kB 65.4 MB/s 
     |████████████████████████████████| 13.4 MB 197 kB/s 
     |████████████████████████████████| 895 kB 57.6 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=b72bef714e3e8e7c680b2f96e80ad223b6b5b7e54029cf871dab45df25340f99
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [2]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [3]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [5]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る。
# 系列長を揃える必要がないので、単にiput_idsのみを入力します。
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [6]:
scores.size()

torch.Size([1, 8, 32000])

In [7]:
masked_position = input_ids[0].tolist().index(4)
# スコアが最も良いトークンのIDを取り出し、トークンに変換する
best_score = max(scores[0][masked_position])
best_index = scores[0][masked_position].tolist().index(best_score)
best_id = scores[0][masked_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(best_id)
token_best = token_best.replace('##', '')
# [MASK]を上で求めたトークンで置き換える。
text = text.replace("[MASK]", token_best)
print(text)

今日は東京へ行く。


In [8]:
def predict_masked_top_k(text, num_top_k, tokenizer, mlm):

    # 文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    # 上位何位まで使うかは、num_topkで指定。
    # 出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
   
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = mlm(input_ids=input_ids)
        scores = output.logits
    # スコアが上位のトークンとスコアを求める。
    masked_position = input_ids[0].tolist().index(4)
    topk = scores[0][masked_position].topk(num_top_k)
    topk_ids = topk.indices
    best_token = tokenizer.convert_ids_to_tokens(topk_ids)
    scores_topk = topk.values.cpu().numpy()

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = []
    for i in range(len(best_token)):
        text_cp = text
        text_cp = text_cp.replace('[MASK]', best_token[i])
        text_topk.append(text_cp)
    
    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_predicted, score_predicted = predict_masked_top_k(text, 10, tokenizer, bert_mlm)
print(*text_predicted, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [15]:
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する。
    """
    # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える。
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

今日はカレーを食べる。そしてカレーをする
